In [ ]:
from utz import *
from IPython.display import HTML, SVG, display

import fiona
import geopandas as gpd
from geopy import distance
import shapefile
from shapely.geometry import Point

import plotly.express as px
import plotly.graph_objects as go

import sys
sys.executable

## Mapbox configs

In [ ]:
token = open(".mapbox-token").read()
mapbox = dict(mapbox=dict(
    style="dark",
    accesstoken=token,
))

In [ ]:
from nj_crashes.crashes import c20
from nj_crashes.sri import get_sri_mp_lls, SRI_DB_PATH
from sqlite3 import connect

In [ ]:
county = 'HUDSON'
cc20 = c20[c20['County Name'] == county]
conn = connect(SRI_DB_PATH)
# cc20sm = cc20[(cc20.SRI != '') & (~cc20.MP.isna())]
cc_lls = get_sri_mp_lls(cc20, conn=conn)
lls_hist = cc_lls[['LON','LAT','Severity']].value_counts().rename('lls_count')
cc_lls = cc_lls.merge(lls_hist.reset_index(), on=['LON', 'LAT', 'Severity'], how='left')
cc_lls['lls_count'] = cc_lls['lls_count'].fillna(0)
cc_lls['radius'] = cc_lls.lls_count.apply(sqrt)
cc_lls

In [ ]:
cc_lls.isna().sum()

In [ ]:
clat = (cc_lls.LAT.min() + cc_lls.LAT.max()) / 2
clon = (cc_lls.LON.min() + cc_lls.LON.max()) / 2
clon, clat

In [ ]:
import datashader as ds
cvs = ds.Canvas(plot_width=1000, plot_height=1000)
agg = cvs.points(cc_lls, x='LON', y='LAT', )
# agg is an xarray object, see http://xarray.pydata.org/en/stable/ for more details
coords_lat, coords_lon = agg.coords['LAT'].values, agg.coords['LON'].values
# Corners of the image, which need to be passed to mapbox
coordinates = [[coords_lon[0], coords_lat[0]],
               [coords_lon[-1], coords_lat[0]],
               [coords_lon[-1], coords_lat[-1]],
               [coords_lon[0], coords_lat[-1]]]

from colorcet import fire
import datashader.transfer_functions as tf
img = tf.shade(agg, cmap=fire)[::-1].to_pil()
img

In [ ]:
severities = list(CrashSeverity.ch2Name.values())

In [ ]:
CrashSeverity.ch2Name

In [ ]:
from njdot.codes import CrashSeverity

def crash_plot(
    cc_lls,
    county,
    types='pif',
    center=None, zoom=11.4,
    lg='tr', lm=0.02,
    colors=None, scale_radii=None, size='radius',
    marker=None,
    legend=None,
    W=1000, H=1000,
    layers=None,
    cluster=None,
    density=None,
    density_radius=20,
    zmaxs=None,
    write=True,
    map='carto-darkmatter',  # 'dark' will load .mapbox-token
    show=False,
    **kwargs
):
    name = f'www/public/plots/njdot/{county.lower()}-{types}-crashes'
    scale_radii = scale_radii or [1, 2, 4]
    colors = colors or ['yellow', 'orange', 'red']
    zmaxs = zmaxs or [20, 10, 1]
    center = center or dict(lon=clon, lat=clat)
    if isinstance(center, list):
        center = dict(lon=center[0], lat=center[1])
    severity_dfs = []
    density_traces = []
    for idx, ((ch, severity), color, scale_radius, zmax) in enumerate(zip(CrashSeverity.ch2Name.items(), colors, scale_radii, zmaxs)):
        if ch not in types:
            continue
        severity_df = cc_lls[cc_lls.Severity == severity]
        severity_df = severity_df.assign(radius=severity_df.radius * scale_radius)
        severity_dfs.append(severity_df)

        if density:
            density_trace = go.Densitymapbox(
                name=f'{severity} density',
                lon=severity_df.LON, lat=severity_df.LAT,
                zmin=0, zmax=zmax,
                radius=density_radius,
                colorscale=[[0, 'black'], [1, color]],
                hoverinfo='none',
                showscale=False,
                legendrank=990+idx,
            )
            density_traces.append(density_trace)
    df = pd.concat(severity_dfs)
    fig = px.scatter_mapbox(
        df,
        lon='LON', lat='LAT',
        size=size,
        #size_max=10,
        color='Severity',
        color_discrete_map=dict(zip(severities, colors)),
        hover_data=['Date', 'Crash Location', 'SRI', 'MP', 'lls_count',],
        center=center,
        zoom=zoom,
        height=H,
        **kwargs
    )
    legend_bgcolor = '50'
    mapbox_kwargs = dict(style=map)
    if map == 'dark':
        token = open(".mapbox-token").read()
        mapbox_kwargs['accesstoken'] = token
    if density_traces:
        for trace in density_traces:
            fig.add_trace(trace)
        fig.data = fig.data[-len(density_traces):] + fig.data[:len(density_traces)]
    if lg == 'tr':
        legend_pos = dict(yanchor="top", xanchor="right", x=1-lm, y=1-lm)
    elif lg == 'br':
        legend_pos = dict(yanchor="bottom", xanchor="right", x=1-lm, y=lm)
    else:
        raise ValueError(f'Unrecognized legend position: {lg}')
    legend_obj = dict(
        title=dict(text=''),
        **legend_pos,
        font=dict(
            size=14,
            color="white"
        ),
        bgcolor=f"rgba({legend_bgcolor},{legend_bgcolor},{legend_bgcolor},0.8)",
        bordercolor="white",
        borderwidth=2,
    )
    if legend:
        legend_obj.update(legend)
    legend = legend_obj
    fig.update_layout(
        mapbox=dict(
            # dict(style=<style>) or dict(style="dark", accesstoken=token)
            **mapbox_kwargs,
            layers=layers,
            #layers=[ { "sourcetype": "image", "source": img, "coordinates": coordinates }, ]
        ),
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        title=dict(
            text=f"{county} County crashes (2020)",
            x=0.5, y=0.98,
            xanchor='center', yanchor='top',
            font=dict(size=32, color="white")
        ),
        legend=legend,
    )
    scatter_kwargs = dict(
        cluster=cluster,
        #marker=marker if marker else None,
    )
    for idx, (severity, color) in enumerate(zip(severities, colors)):
        fig.update_traces(selector=dict(name=severity), legendrank=1001 + idx, marker=dict(color=color), **scatter_kwargs)
    if cluster:
        fig.update_traces(cluster=dict(enabled=True))
    if write is True or write == 'png':
        fig.write_image(f'{name}.png', width=W, height=H)
    if write is True or write == 'json':
        fig.write_json(f'{name}.json')
    if show:
        fig.show(config=dict(displayModeBar=False))
    return fig

In [ ]:
args = (cc_lls, 'Hudson',)
def plot_mix(types='pif', **kwargs):
    kw = dict(
        size_max=10,
        types=types,
        density=True,
        zoom=11.5,
        lg='br',
        H=1100,
    )
    kw.update(**kwargs)
    crash_plot(*args, **kwargs, map='dark', write='png', **kw)
    return crash_plot(*args, **kwargs, write='json', **kw)

In [ ]:
plot_mix('pif')

In [ ]:
plot_mix('if')

In [ ]:
plot_mix('f')

In [ ]:
cc_lls[['Severity', 'radius']].value_counts().sort_index()

In [ ]:
crash_plot(cc_lls, 'Hudson', types='if', size='radius', size_max=10)

In [ ]:
fig = px.density_mapbox(
    cc_lls.sort_values('Severity', ascending=False),
    lon='LON', lat='LAT',
    #size='radius',
    #color='Severity',
    #color_discrete_map={ 'Property Damage': 'yellow', 'Injury': 'orange', 'Fatal': 'red', },
    hover_data=['Date', 'Crash Location', 'SRI', 'MP', 'lls_count',],
    #center=dict(lon=-74.042037, lat=40.725527),
    zoom=11,
    height=1000,
)
legend_bgcolor = '50'
token = open(".mapbox-token").read()
fig.update_layout(
    mapbox=dict(
        style="dark",
        accesstoken=token,
    ),
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    title=dict(
        text=f"{county} County crashes (2020)",
        x=0.5, y=0.98,
        xanchor='center', yanchor='top',
        font=dict(size=32, color="white")
    ),
    legend=dict(
        title=dict(text=''),
        x=0.98, y=0.98,
        xanchor="right", yanchor="top",
        font=dict(
            size=14,
            color="white"
        ),
        bgcolor=f"rgba({legend_bgcolor},{legend_bgcolor},{legend_bgcolor},0.8)",
        bordercolor="white",
        borderwidth=2,
    ),
)
fig.update_traces(selector=dict(name='Property Damage'), legendrank=1001)
fig.update_traces(selector=dict(name='Injury'), legendrank=1002)
fig.update_traces(selector=dict(name='Fatal'), legendrank=1003)
#fig.write_image(path, width=width, height=height)
fig.show()

In [ ]:
from nj_crashes.crashes import c20
from nj_crashes.sri import get_mp_ll, SRI_DB_PATH, SRI_DB_URL, SRI_DB_TABLE

In [ ]:
n = 100
samples = list(reversed(sorted(np.random.exponential(0.5, n))))
print(f'mean: {sum(samples) / len(samples)}')
px.line(samples)

In [ ]:
hc20 = c20[c20['County Name'] == 'HUDSON']
hc20

In [ ]:
pd.crosstab(hc20.SRI != '', ~hc20.MP.isna())

In [ ]:
missing_sri_mp = hc20[(hc20.SRI == '') | hc20.MP.isna()]
missing_sri_mp

In [ ]:
missing_sri_mp['Crash Location'].value_counts()

In [ ]:
nas = missing_sri_mp.replace('', nan).isna()
sxs((~nas).sum().rename('an'), nas.sum().rename('nan'))

In [ ]:
missing_sri_mp['Municipality Name'].value_counts()

In [ ]:
sxs(hc20.SRI.replace('', nan), hc20.MP).isna().sum()

In [ ]:
hc_sris = hc20.SRI.unique()
len(hc_sris), hc_sris

In [ ]:
gc = grove_crashes[['Date', 'Crash Location', 'Severity', 'SRI', 'MP']]
gc

In [ ]:
from sqlite3 import connect
conn = connect(SRI_DB_PATH)

In [ ]:
hc20sm = hc20[(hc20.SRI != '') & (~hc20.MP.isna())]
hc20sm

In [ ]:
sri, mp = hc20sm.SRI.iloc[0], hc20sm.MP.iloc[0]
sri, mp

In [ ]:
get_mp_ll(sri=sri, mp=mp, conn=conn)

In [ ]:
%%time
hc_lls = hc20sm.apply(lambda r: get_mp_ll(sri=r.SRI, mp=r.MP, conn=conn), axis=1)
hc_lls

In [ ]:
grove_crashes.Severity.value_counts()

In [ ]:
gcll = sxs(gc, gc.apply(lambda r: get_mp_ll(r.SRI, r.MP), axis=1).rename('ll'))
gcll['LON'] = gcll.ll.apply(lambda p: p[0])
gcll['LAT'] = gcll.ll.apply(lambda p: p[1])
gcll = gcll.drop(columns='ll')
gcll

In [ ]:
fig = px.scatter_mapbox(
    gcll,
    lon='LON', lat='LAT',
    color='Severity',
    color_discrete_sequence=['yellow', 'orange', 'red'],
    hover_data=['Date', 'Crash Location', 'SRI', 'MP'],
    center=dict(lon=-74.042037, lat=40.725527),
    zoom=13.5,
    height=600,
)
legend_bgcolor = '50'
fig.update_layout(
    **mapbox,
    margin={"r":0,"t":0,"l":0,"b":0},
    title=dict(
        text="Grove St crashes (2020)",
        x=0.5, y=0.98,
        xanchor='center', yanchor='top',
        font=dict(size=32, color="white")
    ),
    legend=dict(
        title=dict(text=''),
        x=0.98, y=0.98,
        xanchor="right", yanchor="top",
        font=dict(
            size=14,
            color="white"
        ),
        bgcolor=f"rgba({legend_bgcolor},{legend_bgcolor},{legend_bgcolor},0.8)",
        bordercolor="white",
        borderwidth=2,
    ),
)
fig.show(config=dict(displayModeBar=False))